In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime

In [2]:
driver = 0
def start_chrome():
    global driver
    options = webdriver.ChromeOptions()
    options.add_argument('user-data-dir=C:\\Users\\Root\\AppData\\Local\\Google\\Chrome\\User Data')
    driver = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
    driver.get("https://www.fxautotrade.com/login")
    driver.find_elements(By.CLASS_NAME, 'btn-google')[0].click()
    time.sleep(5)
    driver.get("https://www.fxautotrade.com/manage/4170848")
    time.sleep(10)
    
start_chrome()

InvalidArgumentException: Message: invalid argument: user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir


In [ ]:
def mt5connect():
    account = int(158690)

    mt5.initialize()
    authorized=mt5.login(account, password="thncjk0z", server="Osprey-Demo")

    if authorized:
        print("Connected: Connecting to MT5 Client")
    else:
        print("Failed to connect at account #{}, error code: {}"
              .format(account, mt5.last_error()))
mt5connect()

In [ ]:
def positions_get(symbol=None):
    if(symbol is None):
	    res = mt5.positions_get()
    else:
        res = mt5.positions_get(symbol=symbol)

    if(res is not None and res != ()):
        df = pd.DataFrame(list(res),columns=res[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        return df
    
    return pd.DataFrame()

def close_position(deal_id):
    open_positions = positions_get()
    
    open_positions = open_positions[open_positions['ticket'] == deal_id]
    order_type  = list(open_positions["type"])[0]
    symbol = list(open_positions['symbol'])[0]
    volume = list(open_positions['volume'])[0]

    if(order_type == mt5.ORDER_TYPE_BUY):
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
	
    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(volume),
        "type": order_type,
        "position": deal_id,
        "price": price,
        "magic": 234000,
        "comment": "Close trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(close_request)
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Failed to close order :(")
    else:
        print ("Order successfully closed!")


def open_position(pair, order_type, size, tp=None, stop=None):
    symbol_info = mt5.symbol_info(pair)
    if symbol_info is None:
        print(pair, "not found")
        return

    if not symbol_info.visible:
        print(pair, "is not visible, trying to switch on")
        if not mt5.symbol_select(pair, True):
            print("symbol_select({}}) failed, exit",pair)
            return
    print(pair, "found!")

    point = symbol_info.point
    
    sl = stop
    tp = tp
        
    if(order_type == "BUY"):
        order = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(pair).ask
        
            
    if(order_type == "SELL"):
        order = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(pair).bid
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": pair,
        "volume": float(size),
        "type": order,
        "price": price,
        "magic": 234000,
        "comment": "",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    if stop:
        request["sl"] = sl
    if tp:
        request["tp"] = tp

    result = mt5.order_send(request)
    #print(result)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Failed to send order :(")
        print(mt5.last_error())
        print(result)
        return -1
    else:
        print ("Order successfully placed!")
        
        
    
    return result

In [ ]:
trades_string = []
class trade:
    def __init__(self, t_id, date, time, side, lot, symbol, o, sl, tp):
        self.t_id = t_id
        self.date = date
        self.time = time
        self.side = side
        self.lot = lot
        self.symbol = symbol
        self.executed = False
        self.closed = False
        
        self.o = None
        self.sl = None
        self.tp = None
        
        if o != "-":
            self.o = float(o)
        if sl != "-":
            self.sl = float(sl)
        if tp != "-":
            self.tp = float(tp)
        

    def enter(self):
        if self.executed:
            print("already executed!")
            return
        
        self.res = open_position(self.symbol,self.side.upper(), max(round(float(self.lot)/5*100)/100, 0.01))
        self.executed = True
        print(self.res.order)
        
    def close(self):
        if self.closed:
            print("already closed")
                                 
        print("close", self.t_id)
        print(close_position(self.res.order))
        self.closed = True
        
    closed = False
    executed = False
        
    t_id = 0
    date = ""
    time = ""
    side = ""
    lot = ""
    symbol = ""
    o = 0
    sl = 0
    tp = 0

    
trades = []

In [ ]:
def update_trades_string():
    global trades_string
    table = driver.find_elements(By.ID, "openTradesCont")[0]
    tr_elements = table.find_elements_by_tag_name("tr")
    trades_string = []
    for i in tr_elements:
        t = i.text
        if "Buy" in t or "Sell" in t:
            trades_string.append(t)

In [ ]:
def parse_strings():
    global trades
    new_trades = []
    for i in trades_string:
        t_id, date, time, side, lot, symbol, o, sl, tp, current, _, _, _ = i.split()
        new_trades.append(trade(t_id, date, time, side, lot, symbol, o, sl, tp))
        
    for new_trade in new_trades:
        trade_there = False
        for old_trade in trades:
            if old_trade.t_id == new_trade.t_id:
                trade_there = True
        
        if not trade_there:
            new_trade.enter()
            trades.append(new_trade)
            
    
    while True:
        sth = False
        for index, old_trade in enumerate(trades):
            trade_still_there = False
            for new_trade in new_trades:
                if old_trade.t_id == new_trade.t_id:
                    trade_still_there = True
                
            if not trade_still_there:
                old_trade.close()
                del trades[index]
                sth = True
                break
        
        if not sth:
            break

In [ ]:
print("starting update loop...")
while True:
    try:
        time.sleep(1)
        update_trades_string()
        parse_strings()
    except KeyboardInterrupt:
        print("break")
        break
    except Exception as e:
        print(repr(e))
        print("restarting...")
        time.sleep(2)
        driver.quit()
        start_chrome()
        mt5connect()
        

In [ ]:
def close_all_trades():
    while len(trades) > 0:
        trades[0].close()
        del trades[0]
        
close_all_trades()

In [ ]:
driver.quit()